In [21]:
import os
import sqlite3
import urllib.request
import pandas as pd
import requests
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [22]:
# Define the file path and URL
db_filename = "northwind.sqlite"
db_path = f"./db/{db_filename}"
db_url = "https://raw.githubusercontent.com/jpwhite3/northwind-SQLite3/main/dist/northwind.db"

# Step 1: Download the database if it doesn't exist
if not os.path.exists(db_path):
    print("Downloading Northwind database...")
    urllib.request.urlretrieve(db_url, db_path)
    print("Download complete.")
else:
    print("Database already exists locally.")

# Step 2: Connect to the SQLite database
conn = sqlite3.connect(db_path)
print("Connected to the Northwind database.")

# Step 3: (Optional) List available tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:")
print(tables)

Database already exists locally.
Connected to the Northwind database.
Tables in the database:
                    name
0             Categories
1        sqlite_sequence
2   CustomerCustomerDemo
3   CustomerDemographics
4              Customers
5              Employees
6    EmployeeTerritories
7          Order Details
8                 Orders
9               Products
10               Regions
11              Shippers
12             Suppliers
13           Territories


In [23]:
# -------------------------------
# Step 2: Load Required Tables
# -------------------------------
customers_df = pd.read_sql("SELECT * FROM Customers", conn)
orders_df = pd.read_sql("SELECT * FROM Orders", conn)
order_details_df = pd.read_sql("SELECT * FROM 'Order Details'", conn)
products_df = pd.read_sql("SELECT * FROM Products", conn)
categories_df = pd.read_sql("SELECT * FROM Categories", conn)
suppliers_df = pd.read_sql("SELECT * FROM Suppliers", conn)

In [24]:
# Define the list of DataFrame names to process
dataframes = ['customers_df', 'orders_df', 'order_details_df', 'products_df', 'categories_df', 'suppliers_df']

# Placeholder for missing values
placeholder = 'Unknown'

# Process each DataFrame
for df_name in dataframes:
    df = globals()[df_name]

    # Check and print null value counts
    nulls = df.isna().sum()
    print(f"\n--- {df_name} ---")
    print("Null values before filling:")
    print(nulls[nulls > 0])

    # Fill NaN values with placeholder
    df = df.fillna(placeholder)

    # Check for duplicates
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        print(f"{duplicate_count} duplicate rows found in {df_name}. Removing them.")
        df = df.drop_duplicates()
    else:
        print("No duplicate rows found.")

    # Save back to the global variable
    globals()[df_name] = df

    # Confirm
    print(f"{df_name}: nulls filled and duplicates handled.")


--- customers_df ---
Null values before filling:
Address        2
City           2
Region         2
PostalCode     3
Country        2
Phone          2
Fax           24
dtype: int64
No duplicate rows found.
customers_df: nulls filled and duplicates handled.

--- orders_df ---
Null values before filling:
ShippedDate        21
ShipPostalCode    172
dtype: int64
No duplicate rows found.
orders_df: nulls filled and duplicates handled.

--- order_details_df ---
Null values before filling:
Series([], dtype: int64)
No duplicate rows found.
order_details_df: nulls filled and duplicates handled.

--- products_df ---
Null values before filling:
Series([], dtype: int64)
No duplicate rows found.
products_df: nulls filled and duplicates handled.

--- categories_df ---
Null values before filling:
Series([], dtype: int64)
No duplicate rows found.
categories_df: nulls filled and duplicates handled.

--- suppliers_df ---
Null values before filling:
Region       1
Fax         16
HomePage    24
dtype: in

In [25]:
# -------------------------------
# Step 3: Build Dimension Tables
# -------------------------------

# dim_customer
dim_customer = customers_df[[
    'CustomerID', 'CompanyName', 'ContactName', 'City', 'Country'
]].copy()

# dim_product (merge categories and suppliers)
dim_product = products_df.merge(categories_df, on='CategoryID', how='left') \
                         .merge(suppliers_df, on='SupplierID', how='left')

dim_product = dim_product[[
    'ProductID', 'ProductName', 'CategoryName', 'CompanyName', 'Country'
]].rename(columns={
    'CompanyName': 'SupplierName',
    'Country': 'SupplierCountry'
})

# dim_date
orders_df['OrderDate'] = pd.to_datetime(orders_df['OrderDate'], format='mixed')
dim_date = orders_df[['OrderID', 'OrderDate']].copy()
dim_date['Year'] = dim_date['OrderDate'].dt.year
dim_date['Month'] = dim_date['OrderDate'].dt.month
dim_date['Day'] = dim_date['OrderDate'].dt.day

# -------------------------------
# Step 4: Build Fact Table
# -------------------------------

fact_sales = order_details_df.merge(
    orders_df[['OrderID', 'CustomerID', 'OrderDate']], 
    on='OrderID', how='left'
).merge(
    products_df[['ProductID', 'ProductName']], 
    on='ProductID', how='left'
)

fact_sales['OrderDate'] = pd.to_datetime(fact_sales['OrderDate'])
fact_sales['RevenueUSD'] = fact_sales['UnitPrice'] * fact_sales['Quantity']

fact_sales = fact_sales[[
    'OrderID', 'CustomerID', 'ProductID', 'OrderDate', 
    'Quantity', 'UnitPrice', 'Discount', 'RevenueUSD'
]]

# -------------------------------
# Done: Preview Outputs
# -------------------------------
print("Fact Table:")
print(fact_sales.head())

print("\ndim_customer:")
print(dim_customer.head())

print("\ndim_product:")
print(dim_product.head())

print("\ndim_date:")
print(dim_date.head())

Fact Table:
   OrderID CustomerID  ProductID  OrderDate  Quantity  UnitPrice  Discount  \
0    10248      VINET         11 2016-07-04        12       14.0       0.0   
1    10248      VINET         42 2016-07-04        10        9.8       0.0   
2    10248      VINET         72 2016-07-04         5       34.8       0.0   
3    10249      TOMSP         14 2016-07-05         9       18.6       0.0   
4    10249      TOMSP         51 2016-07-05        40       42.4       0.0   

   RevenueUSD  
0       168.0  
1        98.0  
2       174.0  
3       167.4  
4      1696.0  

dim_customer:
  CustomerID                         CompanyName         ContactName  \
0      ALFKI                 Alfreds Futterkiste        Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados        Ana Trujillo   
2      ANTON             Antonio Moreno Taquería      Antonio Moreno   
3      AROUT                     Around the Horn        Thomas Hardy   
4      BERGS                  Berglunds snabbköp

In [26]:
# -------------------------------
# Step 1: Load the world cities CSV
# -------------------------------
cities_path = "worldcities.csv"  # Replace with your actual path if needed
world_cities = pd.read_csv(cities_path)

# -------------------------------
# Step 2: Prepare dim_customer and world_cities for join
# -------------------------------
# Create lowercase versions of city and country names for matching
dim_customer['City_lower'] = dim_customer['City'].str.lower()
dim_customer['Country_lower'] = dim_customer['Country'].str.lower()
world_cities['city_lower'] = world_cities['city'].str.lower()
world_cities['country_lower'] = world_cities['country'].str.lower()

# -------------------------------
# Step 3: Define custom fixes for unmatched names
# -------------------------------
city_fix = {
    "bruxelles": "brussels",
    "sao paulo": "são paulo",
    "tsawassen": "vancouver",
    "kobenhavn": "copenhagen",
    "århus": "aarhus",
    "cunewalde": "dresden",
    "frankfurt a.m.": "frankfurt",
    "köln": "cologne",
    "münchen": "munich",
    "torino": "turin",
    "méxico d.f.": "mexico city",
    "stavern": "oslo",
    "warszawa": "warsaw",
    "lisboa": "lisbon",
    "bräcke": "östersund",
    "genève": "geneva",
    "cowes": "southampton",
    "albuquerque": "albuquerque",
    "anchorage": "anchorage"
}

country_fix = {
    "uk": "united kingdom",
    "usa": "united states"
}

# Apply fixes to standardized city and country columns
dim_customer['City_lower'] = dim_customer['City_lower'].replace(city_fix)
dim_customer['Country_lower'] = dim_customer['Country_lower'].replace(country_fix)

# -------------------------------
# Step 4: Join dim_customer with world_cities
# -------------------------------
enriched_customer = dim_customer.merge(
    world_cities,
    left_on=['City_lower', 'Country_lower'],
    right_on=['city_lower', 'country_lower'],
    how='left'
)

# -------------------------------
# Step 5: Finalize the enriched dim_customer
# -------------------------------
enriched_customer = enriched_customer[[
    'CustomerID', 'CompanyName', 'ContactName', 'City', 'Country',
    'admin_name', 'lat', 'lng', 'population'
]].rename(columns={
    'admin_name': 'Region',
    'lat': 'Latitude',
    'lng': 'Longitude',
    'population': 'CityPopulation'
})

# -------------------------------
# Preview
# -------------------------------
print(enriched_customer.head())

  CustomerID                         CompanyName         ContactName  \
0      ALFKI                 Alfreds Futterkiste        Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados        Ana Trujillo   
2      ANTON             Antonio Moreno Taquería      Antonio Moreno   
3      AROUT                     Around the Horn        Thomas Hardy   
4      BERGS                  Berglunds snabbköp  Christina Berglund   

          City  Country            Region  Latitude  Longitude  CityPopulation  
0       Berlin  Germany            Berlin   52.5200    13.4050       4890363.0  
1  México D.F.   Mexico  Ciudad de México   19.4333   -99.1333      21804000.0  
2  México D.F.   Mexico  Ciudad de México   19.4333   -99.1333      21804000.0  
3       London       UK   London, City of   51.5072    -0.1275      11262000.0  
4        Luleå   Sweden        Norrbotten   65.5844    22.1539         49123.0  


In [27]:
# Step 1: CBU API URL for exchange rates (returns JSON)
url = "https://cbu.uz/ru/arkhiv-kursov-valyut/json/"

try:
    # Step 2: Request exchange rate data
    response = requests.get(url)
    data = response.json()

    # Step 3: Extract USD to UZS rate from JSON
    usd_to_uzs_rate = next(
        (float(item['Rate'].replace(',', '')) for item in data if item['Ccy'] == 'USD'),
        None
    )

    if usd_to_uzs_rate:
        print(f"Current USD to UZS rate: {usd_to_uzs_rate}")

        # Step 4: Convert USD revenue to UZS in fact_sales
        fact_sales['RevenueUZS'] = fact_sales['RevenueUSD'] * usd_to_uzs_rate

    else:
        print("USD exchange rate not found in response.")

except Exception as e:
    print("Error fetching exchange rate from CBU API:", e)
    usd_to_uzs_rate = None

# Step 5: Preview
print(fact_sales[['OrderID', 'RevenueUSD', 'RevenueUZS']].head())

Current USD to UZS rate: 12947.07
   OrderID  RevenueUSD    RevenueUZS
0    10248       168.0  2.175108e+06
1    10248        98.0  1.268813e+06
2    10248       174.0  2.252790e+06
3    10249       167.4  2.167340e+06
4    10249      1696.0  2.195823e+07


In [28]:
# Define the DW database file
dwh_path = "northwind_dwh.db"

# Connect to (or create) the DW SQLite database
dwh_conn = sqlite3.connect(dwh_path)

# Ensure enriched_customer_fixed points to the latest enriched customer dimension
enriched_customer_fixed = enriched_customer

# Write DataFrames to the database (replace existing if any)
fact_sales.to_sql("fact_sales", dwh_conn, if_exists="replace", index=False)
dim_product.to_sql("dim_product", dwh_conn, if_exists="replace", index=False)
dim_date.to_sql("dim_date", dwh_conn, if_exists="replace", index=False)
enriched_customer_fixed.to_sql("dim_customer", dwh_conn, if_exists="replace", index=False)

# Optional: Confirm tables are created
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", dwh_conn)
print("✅ Data warehouse schema stored successfully in SQLite:")
print(tables)

✅ Data warehouse schema stored successfully in SQLite:
           name
0    fact_sales
1   dim_product
2      dim_date
3  dim_customer


In [29]:
# Connect to the SQLite DW
conn = sqlite3.connect("northwind_dwh.db")

# OLAP queries

# Connect to the data warehouse SQLite DB
dwh_conn = sqlite3.connect("northwind_dwh.db")

# Load Roll-Up: Revenue by Year and Country
rollup_df = pd.read_sql_query("""
    SELECT d.Year, c.Country, SUM(f.RevenueUZS) AS TotalRevenueUZS
    FROM fact_sales f
    JOIN dim_customer c ON f.CustomerID = c.CustomerID
    JOIN dim_date d ON f.OrderID = d.OrderID
    WHERE d.Year IS NOT NULL
    GROUP BY d.Year, c.Country
    ORDER BY d.Year, TotalRevenueUZS DESC;
""", conn)

# Load Slice & Dice: USA + Beverages
slice_dice_df = pd.read_sql_query("""
    SELECT c.Country, p.CategoryName, d.Year, SUM(f.RevenueUZS) AS Revenue
    FROM fact_sales f
    JOIN dim_customer c ON f.CustomerID = c.CustomerID
    JOIN dim_product p ON f.ProductID = p.ProductID
    JOIN dim_date d ON f.OrderID = d.OrderID
    WHERE c.Country = 'USA' AND p.CategoryName = 'Beverages'
    GROUP BY c.Country, p.CategoryName, d.Year
    ORDER BY d.Year;
""", conn)


drilldown_df = pd.read_sql_query("""
    SELECT d.Year, d.Month, c.Country, p.CategoryName, SUM(f.RevenueUZS) AS MonthlyRevenue
    FROM fact_sales f
    JOIN dim_date d ON f.OrderID = d.OrderID
    JOIN dim_customer c ON f.CustomerID = c.CustomerID
    JOIN dim_product p ON f.ProductID = p.ProductID
    GROUP BY d.Year, d.Month, c.Country, p.CategoryName
""", conn)

top_categories_df = pd.read_sql_query("""
    SELECT p.CategoryName, SUM(f.RevenueUZS) AS TotalRevenue
    FROM fact_sales f
    JOIN dim_product p ON f.ProductID = p.ProductID
    GROUP BY p.CategoryName
    ORDER BY TotalRevenue DESC
    LIMIT 5;
""", conn)

pivot_df = pd.read_sql_query("""
    SELECT c.Country, p.CategoryName, SUM(f.RevenueUZS) AS Revenue
    FROM fact_sales f
    JOIN dim_customer c ON f.CustomerID = c.CustomerID
    JOIN dim_product p ON f.ProductID = p.ProductID
    GROUP BY c.Country, p.CategoryName;
""", conn)

segmentation_df = pd.read_sql_query("""
    SELECT 
        c.CustomerID, c.Country,
        SUM(f.RevenueUZS) AS TotalSpend,
        CASE 
            WHEN SUM(f.RevenueUZS) >= 70000000000 THEN 'High'
            WHEN SUM(f.RevenueUZS) >= 50000000000 THEN 'Medium'
            ELSE 'Low'
        END AS SpendTier
    FROM fact_sales f
    JOIN dim_customer c ON f.CustomerID = c.CustomerID
    GROUP BY c.CustomerID, c.Country;
""", conn)

map_df = pd.read_sql_query("""
    SELECT 
        c.City, 
        c.Country, 
        c.Latitude, 
        c.Longitude, 
        SUM(f.RevenueUZS) AS TotalRevenue
    FROM fact_sales f
    JOIN dim_customer c ON f.CustomerID = c.CustomerID
    WHERE c.Latitude IS NOT NULL AND c.Longitude IS NOT NULL
    GROUP BY c.City, c.Country, c.Latitude, c.Longitude
""", conn)


# Filters
years = pd.read_sql("SELECT DISTINCT Year FROM dim_date WHERE Year IS NOT NULL ORDER BY Year", conn)['Year'].dropna().astype(int).tolist()
countries = pd.read_sql("SELECT DISTINCT Country FROM dim_customer ORDER BY Country", conn)['Country'].dropna().tolist()
categories = pd.read_sql("SELECT DISTINCT CategoryName FROM dim_product ORDER BY CategoryName", conn)['CategoryName'].dropna().tolist()

In [30]:
# Dash App
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Northwind OLAP Dashboard", style={
        'textAlign': 'center', 'fontFamily': 'Segoe UI, sans-serif', 'fontSize': '40px', 'marginBottom': '40px'
    }),

    html.Div([
        html.Label("Filter by Year (Range):", style={'fontWeight': 'bold', 'fontFamily': 'Segoe UI, sans-serif'}),
        dcc.RangeSlider(
            id='year-slider', min=min(years), max=max(years),
            marks={year: str(year) for year in years},
            value=[min(years), max(years)]
        ),
        html.Label("Select Countries:", style={'fontWeight': 'bold', 'fontFamily': 'Segoe UI, sans-serif', 'marginTop': '20px'}),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{"label": c, "value": c} for c in countries],
            value=countries,
            multi=True
        ),
        html.Label("Select Product Categories:", style={'fontWeight': 'bold', 'fontFamily': 'Segoe UI, sans-serif', 'marginTop': '20px'}),
        dcc.Dropdown(
            id='category-dropdown',
            options=[{"label": cat, "value": cat} for cat in categories],
            value=categories,
            multi=True
        )
    ], style={'width': '80%', 'margin': 'auto', 'marginBottom': '40px'}),

    html.H2("Monthly Revenue Over Time", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),
    dcc.Graph(id='monthly-revenue-chart'),
    
    html.H2("Top Product Categories", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),
    dcc.Graph(id='top-categories-chart'),
    
    html.H2("Heatmap: Revenue by Country & Category", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),    
    dcc.Graph(id='pivot-chart', style={'height': '800px'}),
    
    html.H2("Customer Spend Tiers", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),
    dcc.Graph(id='segmentation-chart'),

    html.H2("Top 5 Countries by Revenue", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),
    dcc.Graph(id='top5-stacked-bar', style={'height': '600px'}),

    html.H2("Country Revenue Trends Over Time", style={
        'fontFamily': 'Segoe UI, sans-serif', 'fontSize': '26px', 'marginTop': '40px'
    }),
    dcc.Graph(id='country-line-chart', style={'height': '600px'}),

    html.H2("Revenue in USA for Beverages by Year", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),
    dcc.Graph(id='slice-dice-chart'),
    
    html.H2("Customer Revenue by City", style={
        'fontFamily': 'Segoe UI, sans-serif',
        'fontSize': '26px',
        'marginTop': '40px'
    }),
    dcc.Graph(id='revenue-map', style={'height': '700px'})
])

@app.callback(
    [
        Output('monthly-revenue-chart', 'figure'),
        Output('top-categories-chart', 'figure'),
        Output('pivot-chart', 'figure'),
        Output('segmentation-chart', 'figure'),
        Output('top5-stacked-bar', 'figure'),
        Output('country-line-chart', 'figure'),
        Output('slice-dice-chart', 'figure'),
        Output('revenue-map', 'figure')
    ],
    [
        Input('year-slider', 'value'),
        Input('country-dropdown', 'value'),
        Input('category-dropdown', 'value')
    ]
)
def update_charts(year_range, selected_countries, selected_categories):
    y1, y2 = year_range
    drill = drilldown_df.query("Country in @selected_countries and CategoryName in @selected_categories and @y1 <= Year <= @y2")
    monthly = drill.groupby(['Year', 'Month'])['MonthlyRevenue'].sum().reset_index()
    top = top_categories_df.query("CategoryName in @selected_categories")
    pivot = pivot_df.query("Country in @selected_countries and CategoryName in @selected_categories")
    seg = segmentation_df.query("Country in @selected_countries")

    rollup_filtered = rollup_df.query("@y1 <= Year <= @y2 and Country in @selected_countries")
    rollup_filtered['Year'] = rollup_filtered['Year'].astype(str)

    top5_df = rollup_filtered.groupby('Year', group_keys=False).apply(
        lambda g: g.sort_values('TotalRevenueUZS', ascending=False).head(5)
    )
    top5_fig = px.bar(top5_df, x='Year', y='TotalRevenueUZS', color='Country', barmode='stack')
    top5_fig.update_layout(xaxis_type='category')

    line_fig = px.line(
        rollup_filtered.groupby(['Year', 'Country'])['TotalRevenueUZS'].sum().reset_index(),
        x='Year', y='TotalRevenueUZS', color='Country', markers=True
    )
    line_fig.update_layout(xaxis_type='category')

    sliced = slice_dice_df.query("@y1 <= Year <= @y2")
    sliced['Year'] = sliced['Year'].astype(str)
    geo = map_df.query("Country in @selected_countries")

    fig1 = px.bar(monthly, x=monthly['Year'].astype(str) + '-' + monthly['Month'].astype(str), y='MonthlyRevenue')
    fig2 = px.bar(top, x='CategoryName', y='TotalRevenue')
    fig3 = px.density_heatmap(pivot, x='CategoryName', y='Country', z='Revenue')
    fig4 = px.histogram(seg, x='SpendTier', color='SpendTier', category_orders={"SpendTier": ["Low", "Medium", "High"]})
    fig6 = px.bar(sliced, x='Year', y='Revenue')
    fig7 = px.scatter_geo(geo, lat='Latitude', lon='Longitude', size='TotalRevenue', color='Country', hover_name='City', projection="natural earth", size_max=20)

    return fig1, fig2, fig3, fig4, top5_fig, line_fig, fig6, fig7

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='tab', port=8051)

Dash app running on http://127.0.0.1:8051/


<IPython.core.display.Javascript object>